In [ ]:
from google.colab import files
data=files.upload()

Saving ETTh1.csv to ETTh1 (1).csv
Saving ETTh2.csv to ETTh2.csv
Saving ETTm1.csv to ETTm1.csv
Saving ETTm2.csv to ETTm2.csv


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np

ettH1_data = pd.read_csv('ETTh1 (1).csv')
ettH2_data = pd.read_csv('ETTh2.csv')
ettM1_data = pd.read_csv('ETTm1.csv')
ettM2_data = pd.read_csv('ETTm2.csv')

# Preprocess ETT H1 data
ett_h1_df = ettH1_data[['date', 'HUFL']]
ett_h1_df['date'] = pd.to_datetime(ett_h1_df['date'])
ett_h1_df = ett_h1_df.set_index('date')

# Preprocess ETT H2 data
ett_h2_df = ettH2_data[['date', 'HULL']]
ett_h2_df['date'] = pd.to_datetime(ett_h2_df['date'])
ett_h2_df = ett_h2_df.set_index('date')

# Preprocess ETT M1 data
ett_m1_df = ettM1_data[['date', 'HUFL']]
ett_m1_df['date'] = pd.to_datetime(ett_m1_df['date'])
ett_m1_df = ett_m1_df.set_index('date')

# Preprocess ETT M2 data
ett_m2_df = ettM2_data[['date', 'HULL']]
ett_m2_df['date'] = pd.to_datetime(ett_m2_df['date'])
ett_m2_df = ett_m2_df.set_index('date')

# Scale the data using MinMaxScaler
scaler_h1 = MinMaxScaler()
scaler_h2 = MinMaxScaler()
scaler_m1 = MinMaxScaler()
scaler_m2 = MinMaxScaler()

ett_h1_scaled = scaler_h1.fit_transform(ett_h1_df.values)
ett_h2_scaled = scaler_h2.fit_transform(ett_h2_df.values)
ett_m1_scaled = scaler_m1.fit_transform(ett_m1_df.values)
ett_m2_scaled = scaler_m2.fit_transform(ett_m2_df.values)

def create_lstm_model(sequence_length):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(sequence_length, 1)))
    model.add(LSTM(units=50))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def train_lstm(data):
    sequence_length = 50
    X = []
    y = []
    for i in range(len(data) - sequence_length):
        X.append(data[i:(i + sequence_length), 0])
        y.append(data[i + sequence_length, 0])
    X, y = np.array(X), np.array(y)

    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    model = create_lstm_model(sequence_length)
    model.fit(X, y, epochs=5, batch_size=32)

    predictions = []
    current_sequence = X[-1]
    for i in range(10 * sequence_length):
        prediction = model.predict(current_sequence.reshape(1, sequence_length, 1))[0, 0]
        predictions.append(prediction)
        current_sequence = np.roll(current_sequence, -1)
        current_sequence[-1] = prediction

    return np.array(predictions)

predictions_h1 = train_lstm(ett_h1_scaled)
predictions_h2 = train_lstm(ett_h2_scaled)
predictions_m1 = train_lstm(ett_m1_scaled)
predictions_m2 = train_lstm(ett_m2_scaled)

combined_predictions = np.column_stack((predictions_h1, predictions_h2, predictions_m1, predictions_m2))
print(combined_predictions)


<ipython-input-11-9734b1548409>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ett_h1_df['date'] = pd.to_datetime(ett_h1_df['date'])
<ipython-input-11-9734b1548409>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ett_h2_df['date'] = pd.to_datetime(ett_h2_df['date'])
<ipython-input-11-9734b1548409>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

Epoch 1/5
543/543 [==============================] - 33s 54ms/step - loss: 0.0140
Epoch 2/5
543/543 [==============================] - 30s 55ms/step - loss: 0.0038
Epoch 3/5
543/543 [==============================] - 30s 55ms/step - loss: 0.0030
Epoch 4/5
543/543 [==============================] - 30s 55ms/step - loss: 0.0028
Epoch 5/5
1/1 [==============================] - 0s 24ms/step
Epoch 1/5
543/543 [==============================] - 35s 56ms/step - loss: 0.0042
Epoch 2/5
543/543 [==============================] - 31s 57ms/step - loss: 0.0014
Epoch 3/5
543/543 [==============================] - 31s 57ms/step - loss: 0.0011
Epoch 4/5
543/543 [==============================] - 31s 56ms/step - loss: 0.0011
Epoch 5/5
1/1 [==============================] - 0s 41ms/step
Epoch 1/5
2176/2176 [==============================] - 140s 62ms/step - loss: 0.0027
Epoch 2/5
2176/2176 [==============================] - 134s 61ms/step - loss: 0.0011
Epoch 3/5
2176/2176 [=============================